In [1]:
!pip install transformers==4.37.2 optimum==1.13.2 --quiet
!pip install auto-gptq==0.4.2 --extra-index-url https://huggingface.github.io/auto-gptq/index/whl/cul18/ --quiet
!pip install langchain==0.1.9 --quiet
!pip install sentence-transformers==2.4.0 --quiet
!pip install unstructured --quiet
!pip install pdf2image --quiet
!pip install pdfminer.six==20231228 --quiet
!pip install unstructured-inference --quiet
!pip install faiss-gpu==1.7.2 --quiet
!pip install pikepdf==8.13.0 --quiet
!pip install pypdf==4.0.2 --quiet
!pip install pillow_heif==0.15.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 36.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 17.4 MB/

In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

model_name = "TheBloke/Llama-2-13b-Chat-GPTQ"

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

gen_cfg = GenerationConfig.from_pretrained(model_name)
gen_cfg.max_new_tokens=512
gen_cfg.temperature=0.0000001 # 0.0
gen_cfg.return_full_text=True
gen_cfg.do_sample=True
gen_cfg.repetition_penalty=1.11

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, generation_config=gen_cfg)
llm = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [2]:
from textwrap import fill
from langchain.prompts import PromptTemplate

template = """
[INST] <>
You are an AI assistant, you are truthful, unbiased, and honest in your response.

If you are unsure about an answer, truthfully say "I dont know"
<>

{text}[/INST]
"""
prompt = PromptTemplate(input_variables=["text"], template=template,)
text = "Explain artificial intelligency in a few lines"
result = llm.invoke(prompt.format(text=text))
print(fill(result.strip(), width=100))

[INST] <> You are an AI assistant, you are truthful, unbiased, and honest in your response.  If you
are unsure about an answer, truthfully say "I dont know" <>  Explain artificial intelligency in a
few lines[/INST]  Sure, I'd be happy to explain artificial intelligence (AI) in a few lines!
Artificial intelligence is a field of computer science that focuses on creating intelligent machines
that can perform tasks that typically require human intelligence, such as learning, problem-solving,
and decision-making. AI systems use algorithms and machine learning techniques to analyze data and
make predictions or take actions based on that data. Some examples of AI include natural language
processing, image recognition, and autonomous vehicles. While AI has the potential to revolutionize
many industries and improve our lives in many ways, it also raises important ethical and societal
questions related to privacy, bias, and job displacement.


In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!gdown "https://github.com/muntasirhsn/datasets/raw/main/Solar-System-Wikipedia.pdf"

Downloading...
From: https://github.com/muntasirhsn/datasets/raw/main/Solar-System-Wikipedia.pdf
To: /content/Solar-System-Wikipedia.pdf
100% 4.49M/4.49M [00:00<00:00, 28.3MB/s]


In [7]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.vectorstores.utils import filter_complex_metadata
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

pdf_loader = UnstructuredPDFLoader("/content/Solar-System-Wikipedia.pdf")
pdf_doc = pdf_loader.load()
updated_pdf_doc = filter_complex_metadata(pdf_doc)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
chunked_pdf_doc = text_splitter.split_documents(updated_pdf_doc)
embeddings = HuggingFaceEmbeddings()
db_pdf = FAISS.from_documents(chunked_pdf_doc, embeddings)

prompt_template = """
[INST] <>
Use the following context to Answer the question at the end. Do not use any other information. If you can't find the relevant information in the context, just say you don't have enough information to answer the question. Don't try to make up an answer.

<>

{context}

Question: {question} [/INST]
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
Chain_pdf = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db_pdf.as_retriever(),
    chain_type_kwargs={"prompt": prompt},
)
query = "When was the solar system formed?"
result = Chain_pdf.invoke(query)
print(fill(result['result'].strip(), width=100))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[INST] <> Use the following context to Answer the question at the end. Do not use any other
information. If you can't find the relevant information in the context, just say you don't have
enough information to answer the question. Don't try to make up an answer.  <>  The Solar System
formed 4.568 billion years ago from the gravitational collapse of a region within a large molecular
cloud.[e] This initial cloud was likely several light-years across and probably birthed several
stars.[10] As is typical of molecular clouds, this one consisted mostly of hydrogen, with some
helium, and small amounts of heavier elements fused by previous generations of stars. As the region
that would become the Solar System, known as the pre-solar nebula,[11] collapsed, conservation of
angular momentum caused it to rotate faster. The center, where most of the mass the collected,
became surrounding disc.[10] As the contracting nebula rotated faster, it began to flatten into a
protoplanetary disc with a diamet

In [9]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.vectorstores.utils import filter_complex_metadata
from langchain.text_splitter import RecursiveCharacterTextSplitter

web_loader = UnstructuredURLLoader(urls=["https://en.wikipedia.org/wiki/Solar_System"], mode="elements", strategy="fast", )
web_doc = web_loader.load()
updated_web_doc = filter_complex_metadata(web_doc)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
chunked_web_doc = text_splitter.split_documents(updated_web_doc)
db_web = FAISS.from_documents(chunked_web_doc,embeddings)
Chain_web = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db_web.as_retriever(),
    chain_type_kwargs={"prompt": prompt},
)
query = "When was the solar system formed?"
result = Chain_web.invoke(query)
print(fill(result["result"], width=100))
# Hallucination check
query = "How does the transformer architecture work?"
result = Chain_web.invoke(query)
print(fill(result["result"], width=100))

 [INST] <> Use the following context to Answer the question at the end. Do not use any other
information. If you can't find the relevant information in the context, just say you don't have
enough information to answer the question. Don't try to make up an answer.  <>  Formation and
evolution of the Solar System  The Solar System formed 4.568 billion years ago from the
gravitational collapse of a region within a large molecular cloud.[a] This initial cloud was likely
several light-years across and probably birthed several stars.[13] As is typical of molecular
clouds, this one consisted mostly of hydrogen, with some helium, and small amounts of heavier
elements fused by previous generations of stars.[14]  The Solar System[c] is the gravitationally
bound system of the Sun and the objects that orbit it.[11] It was formed 4.6 billion years ago when
a dense region of a molecular cloud collapsed, forming the Sun and a protoplanetary disc. The Sun is
an ordinary main sequence star that maintai